We need to arange the validation data similar to that of test data for the calibration model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xml
import xml.etree.ElementTree as ET
import cv2
import math
import warnings
from skimage.transform import resize
import os
from scipy import ndimage
import shutil

In [2]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
def create_subwindows_and_counts(image, numpy_folder, stride = 8, kernel_size = 32):
    im_name = image.split(".")[0]
    im_file = im_name + '.npy'
    count_file = im_name + '_density_map.npy'
    # load the image and the count numpy files
    loaded_im_file = np.load(numpy_folder + '/' + im_file)
    loaded_count_file = np.load(numpy_folder + '/' + count_file)
        
    # create the subwindows and counts as follows
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]
    
    density_sums = []
    catch_image = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = loaded_count_file[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)
            sub_window = resize(sub_window, (32, 32,3))
            catch_image.append(sub_window)

    # save the combined subwindows and counts
    return(catch_image,density_sums, im_file)

In [4]:
# we have the density files and the image files as npy files for validation data stored at Preprocessed_valid_data/all_img_density_files/
# let's use those and create the stackers. 

In [5]:
# image and desnity file location for validation data
valid_file_location = 'Preprocessed_valid_data/all_img_density_files'

In [6]:
# contents at this location, and there shapes?
all_valid_files = os.listdir(valid_file_location)
all_valid_files.sort()

In [7]:
for file in all_valid_files:
    loaded_file = np.load(os.path.join(valid_file_location, file))
    print(loaded_file.shape)

(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(1024, 768, 3)
(1024, 768)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)
(768, 1024, 3)
(768, 1024)


In [8]:
# get only the images and not the maps
valid_im_names = [item for item in all_valid_files if item.split(".")[0][-3:] != 'map']

In [9]:
# check the count
len(valid_im_names)

32

In [10]:
%%time
# get the subwindows and counts for test data

im_names = []
for image in valid_im_names:
    val_ims, val_maps, val_names = create_subwindows_and_counts(image, valid_file_location, stride = 8, kernel_size = 32)
    val_im_array = np.array(val_ims)
    val_map_array = np.array(val_maps)
    print(image, val_im_array.shape, val_map_array.shape)
    # save the arrays
    # image stack
    save_path = 'final_valid_sub_windows_and_counts/Block_24'
    np.save(save_path + '/' + 'val_ims_' + image, val_im_array)
    # count stack
    np.save(save_path + '/' + 'val_counts_' + image, val_im_array)
    im_names.append(val_names)

Block0204_2020_06_29.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_01.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_02.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_06.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_07.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_08.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_14.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_16.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_17.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_20.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_22.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_07_23.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_03.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_04.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_06.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_07.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_11.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_12.npy (12288, 32, 32, 3) (12288,)
Block0204_2020_08_14.npy (12288, 32, 32, 3) (1

We also need the true validation counts. Two ways to get this, lets make sure they both give the same results.

In [11]:
# One is by concatenating the stored true values

In [12]:
h_counts_valid = pd.read_csv("True_tassel_counts/all_data/horizontal_annotated_true_counts_valid.csv")
va_counts_valid = pd.read_csv("True_tassel_counts/all_data/vertical_annotated_true_counts_valid.csv")


In [13]:
h_counts_valid.head()

,name,true_count
0,Block0204_2020_08_03,38
1,Block0204_2020_08_04,42
2,Block0204_2020_08_06,39
3,Block0204_2020_08_07,37
4,Block0204_2020_08_11,34


In [14]:
va_counts_valid.head()

,name,true_count
0,Block0204_2020_07_07,2
1,Block0204_2020_07_08,10
2,Block0204_2020_07_14,30
3,Block0204_2020_07_16,33
4,Block0204_2020_07_17,28


In [15]:
h_counts_valid.shape, va_counts_valid.shape

((20, 2), (8, 2))

In [16]:
# note that four of the vn images are missing, we will create a dataframe for that
vn_counts_valid = pd.DataFrame({'name':['Block0204_2020_06_29', 'Block0204_2020_07_01', 'Block0204_2020_07_02', 'Block0204_2020_07_06'], 'true_count': [0,0,0,0]})

In [17]:
vn_counts_valid

,name,true_count
0,Block0204_2020_06_29,0
1,Block0204_2020_07_01,0
2,Block0204_2020_07_02,0
3,Block0204_2020_07_06,0


In [18]:
# okay, lets concatenate these three dataframes together
final_valid_true_counts = pd.concat([vn_counts_valid, va_counts_valid, h_counts_valid], axis = 0)

In [19]:
final_valid_true_counts.head()

,name,true_count
0,Block0204_2020_06_29,0
1,Block0204_2020_07_01,0
2,Block0204_2020_07_02,0
3,Block0204_2020_07_06,0
0,Block0204_2020_07_07,2


In [20]:
final_valid_true_counts.shape

(32, 2)

In [21]:
# verify these counts are correct from the stored density maps

In [22]:
# location of validation density maps
valid_maps_location = 'Preprocessed_valid_data/all_img_density_files/'

In [23]:
contents_here = os.listdir(valid_maps_location)
contents_here.sort()

In [24]:
len(contents_here)

64

In [25]:
# get only the counts maps
valid_count_maps = [file for file in contents_here if file.split('.')[0][-3:] == 'map']

In [26]:
%%time
final_true_counts_npy = []
for item in valid_count_maps:
    loaded_file = np.load(os.path.join(valid_maps_location, item))
    sum_file = np.sum(loaded_file)
    final_true_counts_npy.append(int(np.round(sum_file)))

CPU times: user 41 ms, sys: 85.2 ms, total: 126 ms
Wall time: 380 ms


In [27]:
# final_true_counts_npy

In [28]:
np.mean(final_true_counts_npy == list(final_valid_true_counts['true_count']))

1.0